# Language guessing

In [191]:
import langid
langid.set_languages(["de", "en"])

def guess_language(string_in_language):
    return (langid.classify(string_in_language)[0])

# print(langid.classify("Zur transgenerationalen Traumatisierung"))
# print(langid.classify("Ätiologie und Ansätze für die Therapie"))

# print(langid.classify("Zur transgenerationalen Traumatisierung")[0])
print(guess_language("\"A 'true' artist may draw mountainous seas!\" - Eine Würdigung von Paul Watzlawick zu seinem 100. Geburtstag"))
# print(guess_language("What does it mean for children's development whether and to what extent both parents are employed and they accordingly spend part of the day outside the family? And how should care for young children be structured? Research findings from Developmental Psychology provide some answers. (translated by DeepL)"))

de


# Checking strings for non-letters

In [192]:

language = "EnglishX$X$"
# check if string contains any non-letter character:
if not language.isalpha():
    print("yes")
else:
    print("no")

yes


# Reconciling affiliation with ror

In [193]:
import requests_cache
from datetime import timedelta
# from mappings import geonames_countries
# from mappings import abstract_origins
# from mappings import dd_codes

# for reconciling affiliation strings with ror api:
ROR_API_URL = "https://api.ror.org/organizations?affiliation="

# for getting data about a known id:
ROR_API_LOOKUP_URL = "https://api.ror.org/organizations/"

from mappings import dd_codes

def replace_encodings(text):
    for case in dd_codes:
        text = text.replace(case[0], case[1]) 
    return text

urls_expire_after = {
    # Custom cache duration per url, 0 means "don't cache"
    # f'{SKOSMOS_URL}/rest/v1/label?uri=https%3A//w3id.org/zpid/vocabs/terms/09183&lang=de': 0,
    # f'{SKOSMOS_URL}/rest/v1/label?uri=https%3A//w3id.org/zpid/vocabs/terms/': 0,
}

session = requests_cache.CachedSession(
    ".cache/requests",
    allowable_codes=[200, 404],
    expire_after=timedelta(days=30),
    urls_expire_after=urls_expire_after,
)

def get_ror_id_from_api(affiliation_string):
    # this function takes a string with an affiliation name and returns the ror id for that affiliation from the ror api
    # clean the string to make sure things like "^DDS" are replaced:
    affiliation_string = replace_encodings(affiliation_string)
    #replace_encodings(affiliation_string)
    ror_api_url = ROR_API_URL + affiliation_string
    # make a request to the ror api:
    # ror_api_request = requests.get(ror_api_url)
    # make request to api with caching:
    ror_api_request = session.get(
            ror_api_url, timeout=20
    )
    # if the request was successful, get the json response:
    if ror_api_request.status_code == 200:
        ror_api_response = ror_api_request.json()
        # check if the response has any hits:
        if len(ror_api_response["items"]) > 0:
            # if so, get the item with a key value pair of "chosen" and "true" and return its id:
            for item in ror_api_response["items"]:
                if item["chosen"] == True:
                    return item["organization"]["id"]
        else:
            return None
    else:
        return None
    

    # here is a list of affiliation strings to go through:
affiliation_strings = [
    "Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm",
    "Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm",
    "Sektion Medizinische Psychologie, Universitätsklinikum Ulm",
    "Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm",
    "Psychology School, Hochschule Fresenius ^DDS University of Applied Sciences, Düsseldorf",
    "Fakultät Medizin, MSH Medical School Hamburg ^DDS University of Applied Sciences and Medical University, Hamburg",
    "Fakultät Medizin, MSH Medical School Hamburg ^DDS University of Applied Sciences and Medical University, Hamburg",
    "Department of Child and Adolescent Psychiatry, Psychosomatics and Psychotherapy; LVR Klinikum Essen; University Hospital Essen; University of Duisburg-Essen; Essen",
    "Child and Adolescent Psychiatry/Psychology, Erasmus Medical Center Rotterdam"
]

# use the function to get the ror id for each affiliation string:

# for affiliation_string in affiliation_strings:
#     print(replace_encodings(affiliation_string) + ": " + str(get_ror_id_from_api(affiliation_string)))

# print(replace_encodings("Stimulus ^DDS non psychological interference ^DDL analogy"))
print(replace_encodings("Zagreb Children^D&gt;'s Hospital, Pediatric Clinic"))
print("Test " + replace_encodings("Zagreb Children^D&gt;'s Hospital, Pediatric Clinic") + " more")
print(replace_encodings("p &lt; .05"))
print(replace_encodings(' &lt; '))
print(replace_encodings('haha ^DIF ^DTM'))
print(replace_encodings('Geo^Dffrey Je^Dfferson'))
# print(get_ror_id_from_api("Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm"))

Zagreb Childrenğt;'s Hospital, Pediatric Clinic
Test Zagreb Childrenğt;'s Hospital, Pediatric Clinic more
p &lt; .05
 &lt; 
haha ∞ ™
Geoffrey Jefferson


In [194]:
def utf8len(s):
    return len(s.encode('utf-8'))

print(utf8len("Test string"))

11


# Deprecated: Reconcile with Wikidata

The "reconciler" package can use other reconciliation APIs, too. Wikidata is the default. 
To change the API endpoint, call the reconcile() function with the parameter `reconciliation_endpoint="https..."`

In [195]:
from reconciler import reconcile
import pandas as pd

# A DataFrame with a column you want to reconcile.
test_df = pd.DataFrame(
    {
        "City": ["Rio de Janeiro", "São Paulo", "São Paulo", "Natal"],
        "Country": ["Q155", "Q155", "Q155", "Q155"],
        "Land": ["XD-BR", "XD-BR", "XD-BR", "XD-BR"]
    }
)

# Reconcile against type city (Q515), getting the best match for each item.
reconciled = reconcile(test_df["City"], type_id="TerritorialCorporateBodyOrAdministrativeUnit", property_mapping={"geographicAreaCode": test_df["Land"]}, reconciliation_endpoint="https://lobid.org/gnd/reconcile/")
# reconciled = reconcile(test_df["City"], type_id="Q515")

# save the results to a csv file:
test_df.to_csv("test.csv")
reconciled.to_csv("reconciled.csv")


100%|██████████| 1/1 [00:00<00:00, 6052.39it/s]


In [196]:
# testing with some of our terms and GND Subject Headings:
test_df = pd.DataFrame(
     {
         "TermE": ["Inductive Deductive Reasoning", "Spatial Imagery", "Verbal Comprehension"],
         "TermD": ["Induktiv-deduktives logisches Denken", "Räumliche Bildvorstellung", "Verbales Verständnis"],
     }
    )

reconciled = reconcile(test_df["TermE"], type_id="SubjectHeading", reconciliation_endpoint="https://lobid.org/gnd/reconcile/")

# save the results to a csv file:
test_df.to_csv("test_terms.csv")
reconciled.to_csv("reconciled_terms.csv")

# This is not going to work, our cts just have too different wording compared to their gnd subject headings!




100%|██████████| 1/1 [00:00<00:00, 4815.50it/s]


# Use our authority records for insitutes to reconcile affiliations

At first, I thought it was a good idea to do this by exposing our csv files as reconc apis using this JAVA tool: https://okfnlabs.org/reconcile-csv/

It gives us a reconciliation API endpoint (on http://localhost:8000/reconcile) that one could use with the "reconciler" package, like this:


In [197]:
test_df = pd.DataFrame(
     {
         "Obstname": ["Äpfel", "Birne", "Himbeere"],
     }
    )

reconciled = reconcile(test_df["Obstname"], reconciliation_endpoint="http://localhost:8000/reconcile")
reconciled.to_csv("reconciled_fruit.csv")  


100%|██████████| 1/1 [00:00<00:00, 6615.62it/s]


The above works, but we need to select which part to add to the record. It returns the label that was matched, a score, a match True/False (only True if score 1.0! Maybe we can lower the cutoff to 0.75?), and a type 

Also, this only works with dataframes, so whole tables, whereas we want to reconcile individual strings.

Idea: 
Instead of reconciling with the API, we can just import the CSV of the authority institutes (as a list of dicts) and use fuzzywuzzy to match a given affiliation string. 
[Fuzzywuzzy](https://pypi.org/project/fuzzywuzzy/) ("Fuzzy string matching in python") compares two strings and returns a score. We can use a cutoff of 75% to decide if the string matches the label.



# Import our authority institutes as CSV, use fuzzywuzzy to match a given affiliation string to them

Fuzzywuzzy, if passed a list with sublists for synonyms, also automatically looks in there. 

In [198]:

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import csv

# import csv file with dachlux institutes:
with open('institute_lux.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    # save it in a list:
    dachlux_institutes = list(reader)
    # split string "known_names" into a list of strings on "##":
    for institute in dachlux_institutes:
        institute["known_names"] = institute["known_names"].split(" ## ")
# print("Und die ganze Tabelle:")
# print(dachlux_institutes)


# affiliation_string = "Abteilung für Psychologie, University of Luxembourg, Esch-sur-Alzette"
# expected match: uuid: bfe28ac0-4901-4125-aaa6-c1fb6c644b7a, Centre de Prévention des Toxicomanies (CePT)

def match_local_authority_institutes(string, list):
    # this function takes a string and returns the best match from a list of strings
    # first, get the list of strings:
    # then, get the best match (token_set_ratio seems to be the best scorer for our purposes, so we use that):)):
    # It yields 100% for exact matches. It is also insensitive to word order differences.
    best_match = process.extractOne(string, list, scorer=fuzz.token_set_ratio)
    # return best_match
    # or if i just wanted the value in "uuid":
    return best_match[0].get("uuid")

print("Am besten passt:",match_local_authority_institutes("Fakultät für Geisteswissenschaften, Erziehungswissenschaften und Sozialwissenschaften; Universität Luxemburg", dachlux_institutes))
print("Am besten passt:",match_local_authority_institutes("Department of Behavioral and Cognitive Sciences, University of Luxembourg, Esch-sur-Alzette", dachlux_institutes))



Am besten passt: 79702f56-c49f-4951-bd4b-d1939e52dd46
Am besten passt: c22de612-48f9-4952-9bda-51cea5c88785


To use this in the script, we need the following pseudo-code:

- check if the affiliation string contains a country name
- if yes, check if the country name is in the list of countries (D, A, CH, LUX)
- only then use fuzzywuzzy to match the affiliation string to the list of institutes for that country

If we have separate csv files for each country, we can use the country name to select the right file.


# Use a ror id that we have to look up more data

For authority records, we can use the ror id (that we already looked up using the api with the affiliation parameter) to look up more data about the institute, like the country, the name, the aliases, etc.

In [216]:
# https://api.ror.org/organizations/00tjv0s33

# for getting data about a known id:
from tkinter import N


ROR_API_LOOKUP_URL = "https://api.ror.org/organizations/"

# For research institutes/departments, it makes sense to use ror-id for things that are true for the parent 
# organization as well as for the department: country code, the country name, the city. 

# Trouble is: we have no way to know if a reconciled ror-id is for a department or for its parent organization.
# if sb used their university as the affiliation, we would get the ror-id for the university, 
# but if sb has a department, we still get the ror-id for the university, and we can't say: 
# this is the ror-id for the parent or for this exact suborg. Maybe! There is a way to find out: 
# if it's a full match (not partial) then it is a "sameAs" relationship,
# if it's only a partial match to the org name, it's a "related" or "Child" relationship?.

# The response is a JSON object containing a full ROR record. 
# See [ROR data structure](https://ror.readme.io/docs/ror-data-structure) for details about the fields and values in a ROR record.
# we are interested in:
# acryonyms (list)
# aliases (list)
# country.country_code (string), country.country_name (string)
# addresses[0].city (string), 
# addresses[0].country_geonames_id -> can link to our own geographica's
# addresses[0].geonames_city.id (string), maybe: addresses[0].geonames_city.name (string)
# Allowed external IDs: Crossref Funder ID (FundRef), ISNI, Wikidata. Other external IDs not actively curated include GRID, OrgRef, HESA, UCAS, UKPRN, CNRS.
# external_ids.ISNI.preferred (string), or if "null": .external_ids.ISNI.all (array, or just use the first [0])
# external_ids.Wikidata.preferred (string), or if "null": .external_ids.Wikidata.all (array, or just use the first [0])
# relationships (array) mit jeweils:
#    .id (ror-id mit http-Vorspann der related org)
#    .type (string, eins von "Related", "Successor","Predecessor", "Parent", "Child")
#    .label (string, z.B. "Leibniz-Association")
# we also want the German name or name in other languages, if available. 
# it should be in .labels : labels[0].iso639 = "de", labels[0].label = "Leibniz-Gemeinschaft"
# and we can construct a langstring from it like "Leibniz-Gemeinschaft"@de


def get_ror_authority_data(ror_id):
    ror_api_url = ROR_API_LOOKUP_URL + ror_id
    # make a request to the ror api:
    # ror_api_request = requests.get(ror_api_url)
    # make request to api with caching:
    ror_api_request = session.get(
            ror_api_url, timeout=20
    )
    # if the request was successful, get the json response:
    if ror_api_request.status_code == 200:
        ror_api_response = ror_api_request.json()
        name = ror_api_response["name"]
        for acronym in ror_api_response["acronyms"]:
            if acronym is not None:
                acronym = acronym
            else:
                acronym = None
        city = ror_api_response["addresses"][0]["city"]
        geonames_city = ror_api_response["addresses"][0]["geonames_city"]["id"]
        country_code = ror_api_response["country"]["country_code"]
        country_name = ror_api_response["country"]["country_name"]
        external_ids = ror_api_response["external_ids"]

        return name,acronym, city, geonames_city, country_code, country_name, external_ids
        # check if the response has any hits:
        # if len(ror_api_response["items"]) > 0:
        #     # if so, get the item with a key value pair of "chosen" and "true" and return its id:
        #     for item in ror_api_response["items"]:
        #         if item["chosen"] == True:
        #             return item["organization"]["id"]
        # else:
        #     return None
    else:
        return None
    

# print(get_ror_authority_data("0165gz615")) # ZPID
print(get_ror_authority_data("02qnsw591"))

('Centre for European Economic Research', 'ZEW', 'Mannheim', 2873891, 'DE', 'Germany', {'ISNI': {'preferred': None, 'all': ['0000 0004 0492 4665']}, 'Wikidata': {'preferred': None, 'all': ['Q191206']}, 'GRID': {'preferred': 'grid.13414.33', 'all': 'grid.13414.33'}})


# Feld GRANT migrieren

- Vor allem Unterfeld |n - auftrennen! |i aufheben, und wenn möglich überflüssiges, was keine Nummer ist, wegwerfen.


Ergebnis soll so aussehen (ähnlich wie bei Crossref:, aber mit grant_name ähnoich DataCite, dort heißt es aber grantTitle ) :

```
{
    'funder': 
    {
        'funder_name': 'Sächsische Aufbaubank ^DDS Förder bank ^DDS (SAB)', 'funder_id': None
    }, 
        'grants': 
        [
            {
                'grant_number': '100362999 an YG', 
                'grant_name': None
            }
        ], 
        'funding_note': None
},
{'funder': {'funder_name': 'Institute for Applied Research, Development and Further Education (IAF) at the Catholic University of Applied Sciences in Freiburg', 'funder_id': None}, 'grants': None, 'funding_note': None},
{
    'funder': 
    {
        'funder_name': 'JSPS KAKENHI', 
        'funder_id': None
    }, 
    'grants': [
        {
            'grant_number': '15K00871', 
            'grant_name': None
        }, 
        {
            'grant_number': '18KK0055', 
            'grant_name': None
        }
    ], 
    'funding_note': None}
```

Note: may remove the general "funding_note" field, anywhere else, or move its contents as grant_name to each grant (as OpenAlex does). May rename 

In [1]:
n_strings = ("KND1: 01GI0102, 01GI0420, 01GI0422, 01GI0423, 01GI0429, 01GI0431, 01GI0433, 01GI0434; KNDD: 01GI0710, 01GI0711, 01GI0712, 01GI0713, 01GI0714, 01GI0715, 01GI0716",
             "HO5852/1-1","392443797","01GI1008C","801210010-20",
             "TA 857/3-2", "2016YFC1306800", "81671329", "18ZDA293", "17411969900",
             "20144Y0053", "SHDC12014111", "13dz2260500", "ZH2018QNB19",
             "2018-FX-04, 2013-YJGJ-03", "IIR-1303", 
             "01GL1714A; 01GL1714B; 01GL1714C; 01GL1714D, 01GY1613",
             "15K00871 and 18KK0055", "366/14", "386/14", "100362999 an YG"
             )


def extract_grant_numbers(subfield_n_string):
    # this function takes a string and returns a list of award numbers
    # first, split the string on "," or ";" or "and": (first replacing all semicolons and "ands" with commas)")
    subfield_n_string = subfield_n_string.replace(" and ", ", ")
    subfield_n_string = subfield_n_string.replace(";", ",")
    subfield_n_string = subfield_n_string.split(", ")
    # in each of the returned list elements, remove any substrings that are shorter 
    # than 5 characters (to get rid of things like " for" or "KDL: " YG: " etc.)
    # for element in subfield_n_string:
    #     if len(element) < 5:
    #         subfield_n_string.remove(element)
    # go through all the list elements and replace each with a dict,
    # which has a key "grant_number" and a key "grant_name" (which is None for now):
    for i, element in enumerate(subfield_n_string):
        subfield_n_string[i] = {"grant_number": element, "grant_name": None}
    # return the list of dicts:
    return subfield_n_string

# extract_grant_numbers(n_strings[0])



def build_grant_from_starfield(grantfield):
    # this function takes a string and returns a funder, grant number, grant name, grant holder
    # first, use anything before the first "|" as the funder:
    funder = {"funder_name": grantfield.split("|")[0].strip(), "funder_id": None}
    # then check the rest for a grant number:
    if "|n " in grantfield:
        grants = grantfield.split("|n ")[1].split(" |")[0]
        grants = extract_grant_numbers(grants)
    else:
        grants = None
    # then check the rest for a grant name:
    if "|i " in grantfield:
        funding_info = grantfield.split("|i ")[1]
    else:
        funding_info = None
    # return a dict of the variables:
    return {"funder": funder, "grants": grants, "funding_note": funding_info}

GRANTs = (
"Sächsische Aufbaubank ^DDS Förder bank ^DDS (SAB) |n 100362999 an YG |e Golub, Yulia", 
"German Federal Ministry of Education and Research |n KND: 01GI0102, 01GI0420, 01GI0422, 01GI0423, 01GI0429, 01GI0431, 01GI0433, 01GI0434; KNDD: 01GI0710, 01GI0711, 01GI0712, 01GI0713, 01GI0714, 01GI0715, 01GI0716",
'"German Federal Ministry of Education and Research |n 01GL1714A; 01GL1714B; 01GL1714C; 01GL1714D, 01GY1613 |i AgeDifferent.de: Funding program "Gesund^DDL ein Leben lang"',
"Austrian Science Fund (FWF) |i Open access funding",
"JSPS KAKENHI |n 15K00871 and 18KK0055",
"Institute for Applied Research, Development and Further Education (IAF) at the Catholic University of Applied Sciences in Freiburg |e Eric Pfeifer",
)

print(build_grant_from_starfield(GRANTs[3]))


{'funder': {'funder_name': 'Austrian Science Fund (FWF)', 'funder_id': None}, 'grants': None, 'funding_note': 'Open access funding'}
